
<center> <h1> Data Ingestion and cleaning <br> Pandas I/O and Data Cleaup and Preparation

</h1></center>

In [ ]:
pandas_io_tables= pd.read_html('https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html')
len(pandas_io_tables)

In [ ]:
pandas_io_tables[0]

# read_csv

In [5]:
import pandas as pd
import numpy as np

In [ ]:
%%bash
echo -e  "a, b, c\n1, 3, 4\n11, 44, 55\n4, 5, 23"> sample_file.txxxt

In [ ]:
! ls *.txxxt

In [ ]:
!head sample_file.txxxt

This is a delimiter(comma separated) text file. So we can use pandas.read_csv

In [ ]:
df= pd.read_csv('sample_file.txxxt')
df

#  Earth Lab [Figshare](Figshare.com) repository.
download .csv containing monthly average precipitation for Boulder, CO

In [ ]:
pd.read_csv('https://ndownloader.figshare.com/files/12710618')


read_csv has lot of options around **50** to handle complexity of  noisy csv files.

- CSV file has header missing(use **header = None** or give header names using **names = list of header names** )
- Skip some rows( use **skiprows= list of row number**)
- **na_values** to specify a **[ ] of NA values or a dictionary {column_name:na_value}**

In [ ]:
%%bash
echo -e  "11, 233, 123\n1, 3, 4\n11, 44, 55\n4, 5, 23"> sample_file_no_header.txxxt
cat sample_file_no_header.txxxt

In [ ]:
pd.read_csv('sample_file_no_header.txxxt', header= None)

In [ ]:
pd.read_csv('sample_file_no_header.txxxt', names= list('abc'))

# read_table with regular expressions
example from the book

In [ ]:
%%bash
echo -e '            A         B         C\n
 aaa -0.264438 -1.026059 -0.619500\n
 bbb  0.927272  0.302904 -0.032399\n
 ccc -0.264273 -0.386314 -0.217601\n
 ddd -0.871858 -0.348382  1.100491\n' > sample_file_variable_space.txxxt

In [ ]:
!cat sample_file_variable_space.txxxt

In [ ]:
# What regex we should put in sep
pd.read_table('sample_file_variable_space.txxxt', sep=  )

# or read_csv

In [ ]:
pd.read_csv('sample_file_variable_space.txxxt',sep=)

# Reading Text Files in Pieces

let dowload iris dataset form uci machine learning repository

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data

In [ ]:
!head iris.data

Attribute Information:
   1. sepal length in cm
   2. sepal width in cm
   3. petal length in cm
   4. petal width in cm
   5. class: 
      -- Iris Setosa
      -- Iris Versicolour
      -- Iris Virginica


In [ ]:
%%bash 
columns="sepal_length,sepal_width,petal_length,petal_width,class"
echo "$columns"| cat - iris.data > iris.csv

In [ ]:
!head iris.csv

In [ ]:
!wc -l iris.csv

Let's say this file is big and can't fit in memory. We can use **read_csv** with **chunksize** option.

Typically a summarization is performed in chunk to perform overall summarization.

In [ ]:
chunks = pd.read_csv('iris.csv', chunksize=20)
print('type of chunks is {}'.format(type(chunks)))
total_sepal_length = 0
num_examples = 0
flower_by_type = pd.Series([])
for chunk in chunks:
    #??? check the type of chunk
    total_sepal_length += chunk['sepal_length'].sum()
    num_examples+= chunk.shape[0]
    
total_sepal_length/num_examples

let's verify this

In [ ]:
iris_df = pd.read_csv('iris.csv')
iris_df.describe()

# Writing/converting data
use **to_** methods like
- to_csv
- to_pickle
- to_sql
- to_latex
- to_html
- t_excel
.
.
.

@TODO to_latex functionality

#  Handling JSON data
- Use pd.read_json to read json file
- use inbult json (loads, dumps) to handle json string and python object

In [ ]:
iris_str_data = iris_df.to_json()
iris_str_data


use json(part of python standard library) to handle json related functionality

## Deserialize  str data to python object

In [ ]:
import json
# Let's Deserialize iris_str_data
iris = json.loads(iris_str_data)
print(type(iris))
iris.keys()

## Convert python data to JSON

In [ ]:
iris_json_obj = json.dumps(iris)
print(type(iris_json_obj))

# Binary dataformat
- pickle.
- HDF5(hierarchical data format) Format. 
    + Useful for large amount of data in data science/machine learning project. Allows efficient read and write for small portion of data. Relevent portion of data can be pulled into memory very fast(**some kind of queue mechanism**).

There are libraries in python(h5py) to handle hdf5 files but pandas has **HDFStore** interface.

In [ ]:
# may have install tables. 
#pip install --upgrade tables
iris_h5_store = pd.HDFStore('iris_data.h5')

In [ ]:
iris_h5_store['iris_class'] = iris_df['class'] 
iris_h5_store['iris_class']

In [ ]:
iris_h5_store.close()

To allow searching use foramt = table

In [ ]:
iris_df.to_hdf('iris.h5', 'iris_table', format= 'table')

In [ ]:
pd.read_hdf('iris.h5', 'iris_table', columns= ['sepal_length', 'class'])

In [ ]:
pd.read_hdf('iris.h5', 'iris_table', where = ['sepal_length<5.0'])

In [ ]:
pd.read_hdf('iris.h5', 'iris_table', where = ['index < 10'])

In [ ]:
iris_df.to_hdf('iris.h5', 'iris_table', format= 'table', data_columns=['sepal_length', 'class'])

In [ ]:
pd.read_hdf('iris.h5', 'iris_table', where = ['sepal_length<5.0'])

# Data cleaning
## Handling Missing Data

In pandas missing data is NA(not available)..

- For numeric data, pandas uses the floating-point value NaN (Not a Number) to represent missing data.
- The built-in Python None value is also treated as NA


## APIs for handling NA
- **dropna:** Filter data based on axis labels, with varying thresholds for how much missing data to tolerate.
-  **fillna:** Fill NA with a value of some interpolation method	

## dropna

https://archive.ics.uci.edu/ml/machine-learning-databases/horse-colic/horse-colic.data

In [ ]:
# As we talked earlier csv files have lot of options, can you figure out the option
#so that ? is treated as NA values
colic_horse_df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/horse-colic/horse-colic.data', 
                   sep= '\s+', header= None)
colic_horse_df.sample(10)

In [ ]:
colic_horse_df.dropna?

In [ ]:
colic_horse_df.dropna(axis=0, how = 'any')


**how = all** will drop if all row values or column values are NA

## same as this boolean indexing

In [ ]:
import numpy as np
colic_horse_df[ colic_horse_df.notna().all(axis = 1)]

In [ ]:
# Keep  all the row with  number of na values < 4 ??


# Filling In Missing Data

In [ ]:

np.random.seed(1)
df = pd.DataFrame(np.arange(21).reshape(7, 3))
df.iloc[1:4, 1] = np.nan
df.iloc[-2:, 2] = np.nan
df

In [ ]:
df.fillna({1: 0.5, 2: .5})

In [ ]:
df.fillna(method='ffill', limit=2)

In [ ]:
# fill df with median value of each columns


# Data Transformation

# Dropping duplicates

In [ ]:
df = pd.DataFrame(np.arange(12).reshape(4,3), columns=['col'+str(i) for i in range(1, 4)])
df

In [ ]:
np.random.seed(1)
dup_df =pd.concat([df, df.sample(2)])
dup_df = dup_df.reset_index(drop= True)
dup_df.iloc[4, 0] = 33
dup_df

In [ ]:
dup_df.duplicated(subset=['col1', 'col3'])

In [ ]:
dup_df.drop_duplicates(subset=['col1', 'col3'])

# Discretization and Binning

In [ ]:
# exampel from the book
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
bins = [18, 25, 35, 60, 100]


In [ ]:
cats = pd.cut(ages, bins)
print(cats)
type(cats)

In [ ]:
print(cats.codes)
print(cats.categories)




bin counts

In [ ]:
print(pd.value_counts(cats))

## passing bin names

In [ ]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

# outlier detection
- Unusual dollar amount
- Unusual pixel value in the image

build conditional logic to get boolean index

In [6]:
np.random.seed(1)
data = pd.DataFrame(2+ np.random.randn(10, 4))
data

,0,1,2,3
0,3.624345,1.388244,1.471828,0.927031
1,2.865408,-0.301539,3.744812,1.238793
2,2.319039,1.750630,3.462108,-0.060141
3,1.677583,1.615946,3.133769,0.900109
4,1.827572,1.122142,2.042214,2.582815
5,0.899381,3.144724,2.901591,2.502494
6,2.900856,1.316272,1.877110,1.064231
7,1.732112,2.530355,1.308339,1.603246
8,1.312827,1.154794,1.328754,1.987335
9,0.882690,2.234416,3.659802,2.742044


In [7]:
data.describe()

,0,1,2,3
count,10.000000,10.000000,10.000000,10.000000
mean,2.004181,1.595598,2.493033,1.548796
std,0.909089,0.933614,0.990475,0.902147
min,0.882690,-0.301539,1.308339,-0.060141
25%,1.404016,1.195164,1.573149,0.961331
50%,1.779842,1.502095,2.471902,1.421020
75%,2.728815,2.113469,3.380023,2.373705
max,3.624345,3.144724,3.744812,2.742044


In [8]:
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
0,3.624345,1.388244,1.471828,0.927031
1,2.865408,-0.301539,3.744812,1.238793
2,2.319039,1.750630,3.462108,-0.060141
3,1.677583,1.615946,3.133769,0.900109
5,0.899381,3.144724,2.901591,2.502494
9,0.882690,2.234416,3.659802,2.742044


In [9]:
data[0].quantile(q= .99)

3.559231316367347

In [10]:
data[0][np.abs(data[0]) > data[0].quantile(q= .99)] = data[0].quantile(q= .99)
data

,0,1,2,3
0,3.559231,1.388244,1.471828,0.927031
1,2.865408,-0.301539,3.744812,1.238793
2,2.319039,1.750630,3.462108,-0.060141
3,1.677583,1.615946,3.133769,0.900109
4,1.827572,1.122142,2.042214,2.582815
5,0.899381,3.144724,2.901591,2.502494
6,2.900856,1.316272,1.877110,1.064231
7,1.732112,2.530355,1.308339,1.603246
8,1.312827,1.154794,1.328754,1.987335
9,0.882690,2.234416,3.659802,2.742044


# Computing Indicator/Dummy Variables

converting a categorical variable into a “dummy” or “indicator” matrix

Check the book too

In [18]:
!curl -O http://files.grouplens.org/datasets/movielens/ml-latest-small.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  955k  100  955k    0     0  2912k      0 --:--:-- --:--:-- --:--:-- 2975k


In [19]:
!unzip ml-latest-small.zip


Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [20]:
!head ml-latest-small/movies.csv

movieId,title,genres
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
6,Heat (1995),Action|Crime|Thriller
7,Sabrina (1995),Comedy|Romance
8,Tom and Huck (1995),Adventure|Children
9,Sudden Death (1995),Action


In [21]:
movie_df = pd.read_csv('ml-latest-small/movies.csv', index_col = 0)
movie_df.head()

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [23]:
unique_genre =  pd.unique(movie_df.genres.str.split('|').sum())
unique_genre = set(unique_genre)
unique_genre

{'(no genres listed)',
 'Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'IMAX',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western'}

In [26]:
# build columns for different genre indicator
for g in unique_genre:
    movie_df[g] = movie_df.genres.map(lambda x: g in x.split('|'))


,title,genres,Mystery,Animation,Children,Romance,IMAX,Film-Noir,Drama,Musical,...,War,Thriller,Adventure,Fantasy,Documentary,Comedy,Crime,Western,Horror,Sci-Fi
movieId,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,False,True,True,False,False,False,False,False,...,False,False,True,True,False,True,False,False,False,False
2,Jumanji (1995),Adventure|Children|Fantasy,False,False,True,False,False,False,False,False,...,False,False,True,True,False,False,False,False,False,False
3,Grumpier Old Men (1995),Comedy|Romance,False,False,False,True,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
4,Waiting to Exhale (1995),Comedy|Drama|Romance,False,False,False,True,False,False,True,False,...,False,False,False,False,False,True,False,False,False,False
5,Father of the Bride Part II (1995),Comedy,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,False,True,False,False,False,False,False,False,...,False,False,False,True,False,True,False,False,False,False
193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,False,True,False,False,False,False,False,False,...,False,False,False,True,False,True,False,False,False,False
193585,Flint (2017),Drama,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False


In [27]:
movie_df.head()

,title,genres,Mystery,Animation,Children,Romance,IMAX,Film-Noir,Drama,Musical,...,War,Thriller,Adventure,Fantasy,Documentary,Comedy,Crime,Western,Horror,Sci-Fi
movieId,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,False,True,True,False,False,False,False,False,...,False,False,True,True,False,True,False,False,False,False
2,Jumanji (1995),Adventure|Children|Fantasy,False,False,True,False,False,False,False,False,...,False,False,True,True,False,False,False,False,False,False
3,Grumpier Old Men (1995),Comedy|Romance,False,False,False,True,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
4,Waiting to Exhale (1995),Comedy|Drama|Romance,False,False,False,True,False,False,True,False,...,False,False,False,False,False,True,False,False,False,False
5,Father of the Bride Part II (1995),Comedy,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
